In [55]:
#Perceptron en pytorch (en utilisant juste les tenseurs)

In [56]:
import gzip, numpy, torch

In [5]:
batch_size = 5
nb_epochs = 10
eta = 0.00001

In [73]:
# Chargement des données
((data_train, label_train), (data_test, label_test)) = torch.load(gzip.open('mnist.pkl.gz'))

In [76]:
#tensuer avec size Size([63000, 784]) c'est-a-dire : le nombre de lignes du tenseur est 63000 et le nombre de colonnes du tenseur est 784
# il y a 63 000 images dans l'ensemble de données et chaque image fait 28x28 pixels = 784
#data_train.shape ==> torch.Size([63000, 784])
#tensuer avec size Size([7000, 784]) c'est-a-dire : le nombre de lignes du tenseur est 7000 et le nombre de colonnes du tenseur est 784
# il y a 7000 images dans l'ensemble de données et chaque image fait 28x28 pixels = 784
#data_test.shape ==> torch.Size([7000, 784]) 

In [78]:
# crée une tenseur W de poids avec la taille (784,10) pour connecter les 784 pixels d'entrée d'une image du dataset MNIST à 10 sorties (chiffres de 0 à 9).
w = torch.empty((data_train.shape[1], label_train.shape[1]), dtype=torch.float)
# crée un vecteur de biais b de forme (1,10) pour ajuster les prévisions de chaque classe (0 à 9 pour le dataset MNIST)
b = torch.empty((1, label_train.shape[1]), dtype=torch.float)

In [32]:
#w.shape ==> torch.size([784, 10]) 
#b.shape ==> torch.size([1, 10])

torch.Size([1, 10])

In [36]:
# Initialiser les tenseurs de poids w et le vecteur de biais avec des valeurs aléatoires choisies de manière uniforme entre -0.001 et 0.001
torch.nn.init.uniform_(w, -0.001, 0.001)
torch.nn.init.uniform_(b, -0.001, 0.001)

tensor([[ 6.6027e-04, -2.7672e-04, -7.2359e-04,  2.2989e-04,  7.1416e-04,
         -3.9948e-05,  3.2837e-04, -8.7702e-04,  5.8902e-04,  1.6347e-04]])

In [93]:
#Crée un tableau indices contenant tous les entiers de 0 à 62999.
indices = numpy.arange(len(data_train))
#Cette boucle externe permet de parcourir toutes les données d'entraînement plusieurs fois
for n in range(nb_epochs):
    #Cette ligne mélange les indices des données d'entraînement de manière aléatoire avant chaque époque
    numpy.random.shuffle(indices)
    #Cette boucle interne divise l'ensemble de données en lots de taille batch_size=5. À chaque itération, i prend une valeur correspondant à l'index de début d'un batch.
    for i in range(0, len(data_train), batch_size):
        #Récupère les images du lot (batch) en sélectionnant batch_size=5 données à partir de l'index i
        #torch.Size([5, 784]) il y a 5 images et chaque image fait 28x28 pixels = 784
        x = data_train[indices[i:i+batch_size]]
        #Calcule la sortie du modèle (prédiction) pour les entrées x
        #torch.mm(x, w) : Calcule le produit matriciel entre x et w (les poids du modèle).
        #Ajoute le biais à chaque prédiction
        #torch.Size([5, 10]) y de taille [5, 10] contient les prédictions du modèle pour 5 image
        y = torch.mm(x, w) + b 
        #torch.Size([5, 10]) Extrait les labels pour un groupe de 5 images d'entraînement
        t = label_train[indices[i:i+batch_size]] 
        #Calcul du gradient et t représente les labels réels pour le batch actuel, et y représente les prédictions du modèle pour ce même batch.
        #torch.Size([5, 10])
        grad = (t - y)
        #Mise à jour des poids : x est de forme [5,784] Alors : la forme de x.T est [784,5] et grad est de forme [5,10]
        #Cette opération est un produit matriciel entre la transposée des entrées x.T et le gradient grad. Cela permet de calculer la modification des poids à appliquer.
        w += eta * torch.mm(x.T, grad)
        #Mise à jour du biais : 
        #grad.sum(axis=0) ==> calcule la somme des gradients pour chaque classe (torch.Size([10]))
        #Le biais est ajusté pour chaque classe afin de réduire l'erreur globale du modèle
        b += eta * grad.sum(axis=0)

In [97]:
acc = 0.
for i in range(len(data_test)):
    #torch.Size([1, 784]) il y a une image et chaque image fait 28x28 pixels = 784
    x = data_test[i:i+1]
    #torch.mm(x, w) : Calcule le produit matriciel entre x et w (les poids du modèle).
    #Ajoute le biais à chaque prédiction
    #y de taille [1, 10] contient les prédictions du modèle pour une image
    y = torch.mm(x, w) + b
    #torch.Size([1, 10]) Extrait les labels pour une image d'entraînement
    t = label_test[i:i+1]
    #Cette partie compare les classes prédictes par le modèle aux classes réelles.
    #Cette partie incrémente la variable acc avec le nombre de prédictions correctes dans le batch
    acc += torch.argmax(y, 1) == torch.argmax(t, 1)
print(acc / len(data_test))


tensor([0.8596])


In [57]:
# Perceptron en pytorch (en utilisant les outils de Pytorch)

In [68]:
batch_size = 5 # nombre de données lues à chaque fois
nb_epochs = 10 # nombre de fois que la base de données sera lue
eta = 0.00001 # taux d'apprentissage

In [69]:
# on lit les données
((data_train,label_train),(data_test,label_test)) = torch.load(gzip.open('mnist.pkl.gz'))
# on crée les lecteurs de données
train_dataset = torch.utils.data.TensorDataset(data_train,label_train)
test_dataset = torch.utils.data.TensorDataset(data_test,label_test)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

In [72]:
# on initialise le modèle et ses poids
model = torch.nn.Linear(data_train.shape[1],label_train.shape[1])
torch.nn.init.uniform_(model.weight,-0.001,0.001)
# on initiliase l'optimiseur
loss_func = torch.nn.MSELoss(reduction='sum')
optim = torch.optim.SGD(model.parameters(), lr=eta)

In [70]:
for n in range(nb_epochs):
    # on lit toutes les données d'apprentissage
    for x,t in train_loader:
        # on calcule la sortie du modèle
        y = model(x)
        # on met à jour les poids
        loss = loss_func(t,y)
        loss.backward()
        optim.step()
        optim.zero_grad()

In [71]:
# test du modèle (on évalue la progression pendant l'apprentissage)
acc = 0.
# on lit toutes les donnéees de test
for x,t in test_loader:
    # on calcule la sortie du modèle
    y = model(x)
    # on regarde si la sortie est correcte
    acc += torch.argmax(y,1) == torch.argmax(t,1)
# on affiche le pourcentage de bonnes réponses
print(acc/data_test.shape[0])

tensor([0.8569])
